<a href="https://colab.research.google.com/github/Utu232002/BB84-protocol/blob/main/BB84_encrypted_with_AES_GCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycryptodome
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes
from Crypto.Random.random import randint
from base64 import b64encode, b64decode
!pip install cirq --quiet
import cirq
from random import choices

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
def generate_bytes_from_bits(bits):
    # Join bits into a single string
    bit_string = ''.join(str(bit) for bit in bits)
    # Pad the bit string to make its length divisible by 8
    padded_bit_string = bit_string + '0' * (8 - len(bit_string) % 8)
    # Convert the bit string to bytes
    byte_data = bytes(int(padded_bit_string[i:i+8], 2) for i in range(0, len(padded_bit_string), 8))
    return byte_data

In [ ]:

def QKD(num_bits):
  #Setup
  encode_gates = {0: cirq.I, 1: cirq.X}
  basis_gates = {'Z': cirq.I, 'X': cirq.H}

  qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

  #Alice Chooses Bits and Bases
  alice_key = choices([0, 1], k = num_bits)
  alice_bases = choices(['Z', 'X'], k = num_bits)

  #Alice Creates Qubits
  alice_circuit = cirq.Circuit()

  for bit in range(num_bits):

    encode_value = alice_key[bit]
    encode_gate = encode_gates[encode_value]

    basis_value = alice_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    alice_circuit.append(encode_gate(qubit))
    alice_circuit.append(basis_gate(qubit))

  #Bob chooses a Bases
  bob_bases = choices(['Z', 'X'], k = num_bits)

  bob_circuit = cirq.Circuit()

  for bit in range(num_bits):

    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))

  #Bob Measures Qubits
  bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

  #Bob Creates a Key
  bb84_circuit = alice_circuit + bob_circuit

  sim = cirq.Simulator()
  results = sim.run(bb84_circuit)
  bob_key = results.measurements['bob key'][0]

  final_alice_key = []
  final_bob_key = []

  #Compare Bases
  for bit in range(num_bits):

    if alice_bases[bit] == bob_bases[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])

  #Compare Half their Bits
  num_bits_to_compare = int(len(final_alice_key) * .5)
  if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
    final_alice_key = final_alice_key[num_bits_to_compare:]
    final_bob_key = final_bob_key[num_bits_to_compare:]

    print('\n\nWe can use our keys!')
    print('Alice Key: ', final_alice_key)
    print('Bob Key: ', final_bob_key)

  else:
    print('\n\nEve was listening, we need to use a different channel!')
  # Assuming final_alice_key and final_bob_key are lists of bits
  alice_bytes = generate_bytes_from_bits(final_alice_key)
  bob_bytes = generate_bytes_from_bits(final_bob_key)

  # Print the generated bytes
  print('\nGenerated Bytes for Alice:', alice_bytes)
  print('Generated Bytes for Bob:', bob_bytes)

  return alice_bytes

In [ ]:
unencrypted_string = "Just cracked the code! It all makes sense now."

In [ ]:
def generate_key(password, salt):
    # Derive key from password and salt using scrypt (example)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password)

In [ ]:
# Generate a random nonce
nonce = QKD(512)
# Generate a random salt
salt = QKD(512)
# Derive key from password and salt using scrypt
key = scrypt(unencrypted_string.encode(), salt, 32, N=16384, r=8, p=1)
print(key)




We can use our keys!
Alice Key:  [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]
Bob Key:  [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]

Generated Bytes for Alice: b'\xab\x93\xc6\x8c\xf2\xfa\x07\xb7|\x99\x18|\x9er\x14\x00'
Generated Bytes for Bob: b'\xab\x93\xc6\x8c\xf2\xfa\x07\xb7|\x99\x18|\x9er\x14\x00'


We can use our keys!
Alice Key:  [0, 0, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
def aes_gcm_encrypt(message, key, nonce):
    try:
        # Create AES-GCM cipher
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        # Encrypt the message
        ciphertext, tag = cipher.encrypt_and_digest(message.encode())
        # Return the encrypted message along with salt, nonce, and tag
        return ciphertext, salt, nonce, tag
    except (ValueError, AttributeError) as e:
        print("Error during Decryption:", e)
        return None  # Indicate encryption failure

In [ ]:
def aes_gcm_decrypt(ciphertext, salt, nonce, tag, password):
  try:
      # Create AES-GCM cipher
      cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
      # Decrypt the message and verify the tag
      decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
      # Return the decrypted message
      return decrypted_data.decode()
  except (ValueError, AttributeError) as e:
      print("Error during Decryption:", e)
      return None  # Indicate encryption failur


In [ ]:
password = "MySecretPassword123"
# Encrypt the message
encrypted_message, salt, nonce, tag = aes_gcm_encrypt(unencrypted_string,key, nonce)
print("Encrypted message:", b64encode(encrypted_message).decode())
decrypted_message = aes_gcm_decrypt(encrypted_message, salt, nonce, tag, password)
print("Decrypted message:", decrypted_message)

Encrypted message: 0IbP1udACteT2psj/9C1S7xc1HW7OBbReyOTNvXjNjyLyyg1FS3GQqrTPNsndA==
Decrypted message: Just cracked the code! It all makes sense now.
